**Classical PID tuning**

Here below is provided a classical PID control system tuning. By implementing the allready tuned system on simulink

In [6]:
!pip install --upgrade stable-baselines3

In [1]:
!pip install gymnasium

In [1]:
import pybullet as p
import pybullet_data
import gym
import numpy as np
from gym import spaces
import logging

class DroneClassicalEnv(gym.Env):
    def __init__(self):
        super(DroneClassicalEnv, self).__init__()

        # Connect using DIRECT mode (no GUI)
        self.physicsClient = p.connect(p.GUI)  
        if self.physicsClient < 0:
            raise ValueError("Failed to connect to PyBullet")

        p.setAdditionalSearchPath(pybullet_data.getDataPath())
        p.setGravity(0, 0, -9.8)

        # Load drone
        self.drone = p.loadURDF("/Users/ortol/Desktop/Falco/Local_Control/bullet3-master/data/Quadrotor/quadrotor.urdf", [0, 0, 0], useFixedBase=False)
        if self.drone < 0:
            raise ValueError("Failed to load drone URDF")

        self.action_space = spaces.Box(low=0, high=10, shape=(18,), dtype=np.float32) # Kp, Ki, Kd for (x, y, z), (r, p,y)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(6,), dtype=np.float32) # x, y, z, roll, pitch, yaw

        self.target_position = np.array([0, 0, 5])
        self.target_orn = np.array([0, 0, 0, 1])
        self.state = np.zeros(6) # x, y, z, roll, pitch, yaw
        self.max_torque = 2
        self.min_torque = -2
        self.torque = np.zeros(4) # 4 motors
        self.prev_error_pos = np.zeros(3)
        self.prev_error_orn = np.zeros(3)
        self.Ts = 0.01
        self.k1 = 0.001
        self.k2 = 1
        self.prev_prime_pitch_error_orn = 0 
        self.prev_prime_roll_error_orn = 0
        self.m = 3.7695
        self.g = 9.8
        self.max_thrust = 500
        self.min_thrust = -500

        # Mixer matrix for quadrotor (assuming '+' configuration)
        self.mixer_matrix = np.array([
            [1,  1,  1,  1],  # Total thrust
            [1, -1, -1,  1],  # Roll torque
            [1,  1, -1, -1],  # Pitch torque
            [1, -1,  1, -1]   # Yaw torque
        ])

    def step(self, action):
        action = np.array(action).reshape(3, 6)
        Kp = action[0][:]
        Ki = action[1][:]
        Kd = action[2][:]

        Kp_pos = Kp[0:3] # Kp gain for (x, y, z)
        Ki_pos = Ki[0:3] # Ki gain for (x, y, z)
        Kd_pos = Kd[0:3] # Kp gain for (x, y, z)

        Kp_orn = Kp[3:6] # Kp gain for (r, p, y)
        Ki_orn = Ki[3:6] # Ki gain for (r, p, y)
        Kd_orn = Kd[3:6] # Kd gain for (r, p, y)

        pos, orn = p.getBasePositionAndOrientation(self.drone)
        euler = p.getEulerFromQuaternion(orn)
        vel, ang_vel = p.getBaseVelocity(self.drone)

        # Log the retrieved position and orientation
        logging.info(f"Position: {pos}, Orientation: {orn}, Euler: {euler}")

        curr_error_pos = self.target_position - np.array(pos)
        curr_error_orn = np.array(p.getEulerFromQuaternion(self.target_orn)) - np.array(euler)

        # PID controller algorithm for attitude control using a antiwindup PID controller
        curr_pitch_error_orn = curr_error_orn[1]
        prev_pitch_error_orn = self.prev_error_orn[1]
        curr_prime_pitch_error_orn = self.prev_prime_pitch_error_orn + curr_pitch_error_orn - prev_pitch_error_orn - self.k1 * self.k2 * self.Ts
        tau_pitch = Kp_orn[1] * curr_pitch_error_orn + Ki_orn[1] * curr_prime_pitch_error_orn + Kd_orn[1] * (curr_pitch_error_orn - prev_pitch_error_orn) / self.Ts

        # PID controller algorithm for roll control using a antiwindup PID controller as before
        curr_roll_error_orn = curr_error_orn[0]
        prev_roll_error_orn = self.prev_error_orn[0]
        curr_prime_roll_error_orn = self.prev_prime_roll_error_orn + curr_roll_error_orn - prev_roll_error_orn - self.k1 * self.k2 * self.Ts
        tau_roll = Kp_orn[0] * curr_roll_error_orn + Ki_orn[0] * curr_prime_roll_error_orn + Kd_orn[0] * (curr_roll_error_orn - prev_roll_error_orn) / self.Ts

        # PD controller algorithm for z control
        errZ = curr_error_pos[2]
        thrust_z = Kp_pos[2] * errZ + Kd_pos[2] * (errZ - self.prev_error_pos[2]) / self.Ts
        tot_thrust_z = thrust_z - self.m * self.g

        saturated_tau_pitch = np.clip(tau_pitch, self.min_torque, self.max_torque)
        saturated_tau_roll = np.clip(tau_roll, self.min_torque, self.max_torque)
        saturated_thrust_z = np.clip(tot_thrust_z, self.min_thrust, self.max_thrust)

        # Calculate the desired motor thrusts using the mixer matrix
        desired_forces = np.array([saturated_thrust_z, saturated_tau_roll, saturated_tau_pitch, 0])
        motor_thrusts = np.linalg.solve(self.mixer_matrix, desired_forces)
        
        p.applyExternalForce(self.drone, -1, [0, 0, saturated_thrust_z], [0, 0, 0], p.WORLD_FRAME)

        # Apply torques for roll, pitch, and yaw control
        p.applyExternalTorque(self.drone, -1, [saturated_tau_roll, saturated_tau_pitch, 0], p.WORLD_FRAME)
        p.stepSimulation()

        self.state = np.hstack((pos, euler))
        reward = -np.linalg.norm(curr_error_pos) - np.linalg.norm(curr_error_orn)
        done_pos = np.linalg.norm(curr_error_pos) < 0.1
        done_orn = np.linalg.norm(curr_error_orn) < 0.1
        done = done_pos and done_orn

        self.prev_error_pos = curr_error_pos
        self.prev_error_orn = curr_error_orn
        self.prev_prime_pitch_error_orn = curr_prime_pitch_error_orn
        self.prev_prime_roll_error_orn = curr_prime_roll_error_orn

        truncated = False  # Set this based on episode length if needed

        # Log the state and other details
        logging.info(f"Step: State: {self.state}, Reward: {reward}, Done: {done}")

        return self.state, reward, done, truncated, {}

    def reset(self, seed=None, options=None):
        """Reset drone to initial position."""
        super().reset(seed=seed)
        p.resetBasePositionAndOrientation(self.drone, [0, 0, 0], [0, 0, 0, 1])
        self.state = np.zeros(6)
        p.stepSimulation()  # Ensure simulation updates in GUI
        info = {}  # Additional info dictionary
        return self.state, info
    
    def render(self, mode="human"):
        """Refresh the PyBullet GUI."""
        p.stepSimulation()  # Ensure simulation updates in GUI

    def close(self):
        p.disconnect()


pybullet build time: Mar 18 2025 08:09:30


In [16]:
DroneClassicalEnv().reset()

/Users/ortol/miniforge3/lib/python3.12/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [2]:
# Train PPO Agent with detailed logging
from stable_baselines3 import PPO
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

env = DroneClassicalEnv()
env.reset()
print("Environment is ready")
model = PPO("MlpPolicy", env, verbose=1)

try:
    model.learn(total_timesteps=100000)
    model.save("drone_pid_ppo")
except Exception as e:
    logging.error(f"An error occurred: {e}")

Environment is ready
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------
| time/              |      |
|    fps             | 8254 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 5252         |
|    iterations           | 2            |
|    time_elapsed         | 0            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0021731216 |
|    clip_fraction        | 0.00156      |
|    clip_range           | 0.2          |
|    entropy_loss         | -25.6        |
|    explained_variance   | 0            |
|    learning_rate        | 0.0003       |
|    loss                 | 2.62e+03     |
|    n_updates            | 10           |
|    policy_gradient_loss | -0

In [2]:
from stable_baselines3 import PPO
import time
import logging
import numpy as np

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

# Load the trained model
model = PPO.load("drone_pid_ppo")

# Create environment
env = DroneClassicalEnv()
#env.reset()

# Run simulation using the trained model
obs, _ = env.reset()  # Unpack the observation and info
for step in range(1000):  # Simulate for 1000 steps
    action, _states = model.predict(np.array(obs))  # Use the trained model to get actions
    obs, reward, done, info, _ = env.step(action)  # Apply the action
    
    # Log the step details
    logging.info(f"Step: {step}, Action: {action}, Reward: {reward}, Done: {done}, States: {obs}")
    
    env.render()  # Show drone movement
    time.sleep(0.1)



2025-03-23 22:04:37,825 - Position: (0.0, 0.0, -0.0001701388888888889), Orientation: (0.0, 0.0, 0.0, 1.0), Euler: (0.0, -0.0, 0.0)
2025-03-23 22:04:37,825 - Step: State: [ 0.          0.         -0.00017014  0.         -0.          0.        ], Reward: -5.000170138888889, Done: False
2025-03-23 22:04:37,826 - Step: 0, Action: [0.         0.         0.5321883  1.0826473  1.8320225  0.
 0.01793964 0.         0.         0.53887117 0.8967309  0.
 0.         0.         0.9463935  0.         0.         0.70865893], Reward: -5.000170138888889, Done: False, States: [ 0.          0.         -0.00017014  0.         -0.          0.        ]


Version = 4.1 Metal - 89.3
Vendor = Apple
Renderer = Apple M3
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


2025-03-23 22:04:38,032 - Position: (-1.0668818810654268e-26, -2.6038328505259013e-27, 0.02944930429020311), Orientation: (-5.932035973923189e-09, -9.871450286691841e-09, 0.0, 0.9999999999999999), Euler: (-1.186407194784638e-08, -1.974290057338368e-08, 1.171155964309008e-16)
2025-03-23 22:04:38,033 - Step: State: [-1.06688188e-26 -2.60383285e-27  2.94493043e-02 -1.18640719e-08
 -1.97429006e-08  1.17115596e-16], Reward: -4.9705507187432145, Done: False
2025-03-23 22:04:38,033 - Step: 1, Action: [0.05725899 0.         0.3651412  0.53797364 0.         0.
 0.66708994 0.90343827 0.         0.43163177 1.0420353  0.05134857
 1.370273   0.5618065  0.426081   1.7432168  0.         0.17470393], Reward: -4.9705507187432145, Done: False, States: [-1.06688188e-26 -2.60383285e-27  2.94493043e-02 -1.18640719e-08
 -1.97429006e-08  1.17115596e-16]
2025-03-23 22:04:38,227 - Position: (-2.7922601450051844e-26, -1.8289320772428062e-26, 0.05581837216798404), Orientation: (-1.90757738036082e-08, -4.26589630

KeyboardInterrupt: 